In [ ]:
pserver.stop()

In [1]:
from parameter_server import *
pserver = ParameterServer()

Parameter server started on ('127.0.0.1', 9011)
Serving on ('127.0.0.1', 14001)
Sending registering as listener request
Unhandled command [/requestListenerInfo] ~ 16987
Register listener 127.0.0.1:9010
Unhandled command [/registerListener] ~ 127.0.0.1


# Usage of DiskBufferWriter

In [2]:
from disk_buffer_writer import *

In [3]:
background = DiskBufferWriter('background','background.json', 'bin/')
background.expose_to_network(pserver)

In [4]:
import random, math

background.data = [random.random() for i in range(3)]

# Cache to compensate for possible time writing overlaps

In [5]:
dataWriter = DiskBufferWriter('sine_data','output.json', 'bin/')
dataWriter.expose_to_network(pserver)
dataWriter.allow_cache(10)

In [6]:
import random, math

numpoints = random.randint(4,20)

rnums = [random.random() for i in range(15)]
rnums.sort()
dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                   'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                  }

In [7]:
dataWriter.data

{'random': [-0.9770040723827793,
  -0.8448583900966871,
  -0.2418304971064873,
  -0.04960813582924506,
  0.027555768521134283,
  0.24885962660507754,
  0.31157736790164114,
  0.3731815374697267,
  0.5226733090147695,
  0.6501898391541676,
  0.6840986704770886,
  0.7407903133471769,
  0.8007248618100913,
  0.881812595144648,
  0.9631392160538561],
 'sines': [0.07218101149286545,
  0.4683229381535427,
  0.6887278263546774,
  0.15521842581315412,
  -0.0864609130116064,
  -0.7045689760236122,
  -0.8298557824304023,
  -0.9216782480413929,
  -0.9974641945508632,
  -0.8907356070388063,
  -0.8373586271501371,
  -0.7272667595824494,
  -0.5859414197218819,
  -0.36282406048928983,
  -0.11554292496627631]}

In [8]:
import time
for i in range(20):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.1)

Now let's send changes really fast to test disk caching. (It may fail on your system if your drive access is not fast enough, causing the app to crash)

In [9]:
import time
dataWriter.allow_cache(100)

for i in range(200):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.001)

This might leave a large cache, so cleanup:

In [10]:
dataWriter.cleanup()

If you need guaranteed safety, use file locking:

In [11]:

# No cache but file locking
dataWriter.allow_cache(-1)
dataWriter.use_file_lock()

import time
for i in range(10000):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.00001) # Very fast updates

If this is still not enough, you can have the data writer wait for read acknowledge from app.

Currently not implemented.

In [12]:
pserver.stop()

Closed command server
Closed parameter server
